In [5]:
# Import findspark and initialise. 
import findspark
findspark.init()

In [6]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [9]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
#Add the CSV file to the Spark context 
spark.sparkContext.addFile(url)
# Read the CSV file into a DataFrame
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

In [13]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")


In [14]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

# SQL query to find the average price for a four-bedroom house sold in each year
query = """
SELECT 
    year(date) as year,
    ROUND(AVG(price), 2) as average_price
FROM 
    home_sales
WHERE 
    bedrooms = 4
GROUP BY 
    year(date)
ORDER BY 
    year
"""

# Execute the query
average_price_df = spark.sql(query)

# Show the results
average_price_df.show()

+----+-------------+
|year|average_price|
+----+-------------+
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
|2022|    296363.88|
+----+-------------+



In [20]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

# SQL query to find the average price of homes built each year with 3 bedrooms and 3 bathrooms

query_2 = """
SELECT 
    date_built as year_built,
    ROUND(AVG(price), 2) as average_price
FROM 
    home_sales
WHERE 
    bedrooms = 3 AND
    bathrooms = 3
GROUP BY 
    date_built
ORDER BY 
    year_built
"""

# Execute the query
average_price_2_df = spark.sql(query_2)

# Show the results
average_price_2_df.show()


+----------+-------------+
|year_built|average_price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [21]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

# SQL query to find the average price of homes built each year with 3 bedrooms, 3 bathrooms, two floors, and >= 2,000 sqft
query_3 = """
SELECT 
    date_built as year_built,
    ROUND(AVG(price), 2) as average_price
FROM 
    home_sales
WHERE 
    bedrooms = 3 AND
    bathrooms = 3 AND
    floors = 2 AND
    sqft_living >= 2000
GROUP BY 
    date_built
ORDER BY 
    year_built
"""

# Execute the query
average_price_3_df = spark.sql(query_3)

# Show the results
average_price_3_df.show()

+----------+-------------+
|year_built|average_price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [22]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()
# SQL query to find the "view" rating for the average price of homes >= $350,000
query_4 = """
SELECT 
    view,
    ROUND(AVG(price), 2) as average_price
FROM 
    home_sales
WHERE 
    price >= 350000
GROUP BY 
    view
ORDER BY 
    average_price DESC
"""

# Execute the query
view_rating_df = spark.sql(query_4)

# Show the results
view_rating_df.show()

# print the runtime
print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|  91|   1137372.73|
|  97|   1129040.15|
|  84|   1117233.13|
|  75|   1114042.94|
|  89|   1107839.15|
|  78|   1080649.37|
|  77|   1076205.56|
|  87|    1072285.2|
|  86|   1070444.25|
|  82|    1063498.0|
|  90|   1062654.16|
|  99|   1061201.42|
|  76|   1058802.78|
|  85|   1056336.74|
|  95|    1054325.6|
|  98|   1053739.33|
|  81|   1053472.79|
|  83|   1033965.93|
|  94|    1033536.2|
|  88|   1031719.35|
+----+-------------+
only showing top 20 rows

--- 0.5082089900970459 seconds ---


In [23]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

In [24]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [25]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

# Execute the same query using cached data
view_rating_cached_df = spark.sql(query_4)

# Show the results
view_rating_cached_df.show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|average_price|
+----+-------------+
|  91|   1137372.73|
|  97|   1129040.15|
|  84|   1117233.13|
|  75|   1114042.94|
|  89|   1107839.15|
|  78|   1080649.37|
|  77|   1076205.56|
|  87|    1072285.2|
|  86|   1070444.25|
|  82|    1063498.0|
|  90|   1062654.16|
|  99|   1061201.42|
|  76|   1058802.78|
|  85|   1056336.74|
|  95|    1054325.6|
|  98|   1053739.33|
|  81|   1053472.79|
|  83|   1033965.93|
|  94|    1033536.2|
|  88|   1031719.35|
+----+-------------+
only showing top 20 rows

--- 0.8349189758300781 seconds ---


In [36]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
output_path = "./parquet_files"  # replace with your desired output path


In [37]:
# 11. Read the formatted parquet data.
df.write.mode("overwrite").partitionBy("date_built").parquet(output_path)


In [38]:
# 12. Create a temporary table for the parquet data.
parquet_df = spark.read.parquet(output_path)

# Create a temporary view of the Parquet DataFrame
parquet_df.createOrReplaceTempView("home_sales_parquet")

In [39]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

start_time = time.time()

# Execute the query using the Parquet DataFrame
query_view_rating_parquet = """
SELECT 
    view,
    ROUND(AVG(price), 2) as average_price
FROM 
    home_sales_parquet
WHERE 
    price >= 350000
GROUP BY 
    view
ORDER BY 
    average_price DESC
"""

view_rating_parquet_df = spark.sql(query_view_rating_parquet)

# Show the results
view_rating_parquet_df.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|  91|   1137372.73|
|  97|   1129040.15|
|  84|   1117233.13|
|  75|   1114042.94|
|  89|   1107839.15|
|  78|   1080649.37|
|  77|   1076205.56|
|  87|    1072285.2|
|  86|   1070444.25|
|  82|    1063498.0|
|  90|   1062654.16|
|  99|   1061201.42|
|  76|   1058802.78|
|  85|   1056336.74|
|  95|    1054325.6|
|  98|   1053739.33|
|  81|   1053472.79|
|  83|   1033965.93|
|  94|    1033536.2|
|  88|   1031719.35|
+----+-------------+
only showing top 20 rows

--- 0.7775187492370605 seconds ---


In [40]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [41]:
# 15. Check if the home_sales is no longer cached
home_sales_is_cached = spark.catalog.isCached("home_sales")
print(f"Is 'home_sales' cached? {home_sales_is_cached}")


Is 'home_sales' cached? False
